In [1]:
import bldw
import numpy as np
import blimpy as bl
import pandas as pd
import matplotlib.pyplot as plt
import os
%matplotlib inline

/opt/conda/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 80 from C header, got 88 from PyObject
  return f(*args, **kwds)
/opt/conda/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 80 from C header, got 88 from PyObject
  return f(*args, **kwds)
/opt/conda/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/conda/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/conda/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got

numexpr.utils   INFO     Note: NumExpr detected 40 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
numexpr.utils   INFO     NumExpr defaulting to 8 threads.


In [3]:
file = '/datax/scratch/benjb/bliss_LSCX_test/spliced_blc4041424344454647_guppi_58885_63646_MESSIER081_0057.rawspec.0000.h5'
fb = bl.Waterfall(file, load_data=False)
fb.info()


--- File Info ---
DIMENSION_LABELS : [b'time' b'feed_id' b'frequency']
        az_start :                              0.0
       data_type :                                1
            fch1 :                2251.46484375 MHz
            foff :      -2.7939677238464355e-06 MHz
      machine_id :                               20
           nbits :                               32
          nchans :                        536870912
            nifs :                                1
     source_name :                       MESSIER081
         src_dej :                       69:03:55.8
         src_raj :                      9:55:33.192
    telescope_id :                                6
           tsamp :               18.253611007999982
   tstart (ISOT) :          2020-02-06T17:40:46.000
    tstart (MJD) :                58885.73664351852
        za_start :                              0.0

Num ints in file :                               16
      File shape :               (16, 1, 53

In [5]:
print(2**20)

1048576


In [2]:
df0 = pd.read_csv('/datax/scratch/benjb/bl_nearby_stars/star_cadences_catalog_no_band.csv')
arr = np.transpose(np.array([df0['Cadence ID'].values, df0['.h5 path'].values], dtype=str))
u = np.unique(arr, axis=0)
print(len(arr))
print(len(u))

282858
234366


In [3]:
df = pd.read_csv('/datax/scratch/benjb/bl_nearby_stars/star_cadences_catalog_no_band.csv')
df

,Target,Distance,Cadence ID,Frequency,.h5 path
0,GJ1002,4.69,3635,1926,/datag/pipeline/AGBT16A_999_219/holding/splice...
1,GJ1002,4.69,3635,1926,/datag/pipeline/AGBT16A_999_219/holding/splice...
2,GJ1002,4.69,3635,1926,/datag/pipeline/AGBT16A_999_219/holding/splice...
3,GJ1002,4.69,3635,1926,/datag/pipeline/AGBT16A_999_219/holding/splice...
4,GJ1002,4.69,3635,1926,/datag/pipeline/AGBT16A_999_219/holding/splice...
...,...,...,...,...,...
282853,SO0253,3.85,421558,7876,/datag/pipeline/AGBT23B_999_14/blc76_blp36/blc...
282854,SO0253,3.85,421558,7876,/datag/pipeline/AGBT23B_999_14/blc76_blp36/blc...
282855,SO0253,3.85,421558,7876,/datag/pipeline/AGBT23B_999_14/blc76_blp36/blc...
282856,SO0253,3.85,421558,7876,/datag/pipeline/AGBT23B_999_14/blc76_blp36/blc...


In [5]:
band_list = []
missing_files = []

cx_overlap = np.array([8438.96484375, 8251.46484375, 8063.96484375, 7876.46484375, 7688.96484375])

for i, h5 in enumerate(df['.h5 path'].values):
    if i < 38108:
        continue
    print(h5)
    if not os.path.exists(h5):
        missing_files.append(h5)
        continue
    fb = bl.Waterfall(h5, load_data=False)
    f = fb.header['fch1']
    node = os.path.basename(h5)[:5]
    if 'spliced' in h5:
        avg_freq = f + (fb.header['foff']*fb.header['nchans']/2)
        bands = np.array([['L',1500],['S',2400],['C',6000],['X',9375]])
        band = bands[:,0][np.argmin(np.abs(bands[:,1].astype('float')-avg_freq))]
    else:
        if (f > np.min(cx_overlap)) & (f < np.max(cx_overlap)):
            if f in cx_overlap:
                a = np.where(cx_overlap == f)[0]
                if a == int(node[-1]):
                    band = 'C'
                elif a == int(node[-1])-3:
                    band = 'X'
                else:
                    band = 'F' # meant to catch edge cases
            else:
                band = 'F'
        elif (np.abs((f-1500)/187.5) < 5) & (np.abs((f-1500)/187.5) % 1 < 0.01):
            band = 'L'
        elif np.abs((f-2400)/187.5) % 1 < 0.01:
            band = 'S'
        elif (np.abs((f-1500)/187.5) > 5) & (np.abs((f-6000)/187.5) % 1 < 0.01):
            band = 'C'
        elif np.abs((f-9375)/187.5) % 1 < 0.01:
            band = 'X'
    print(i, f, band, node)
    print(h5)
    band_list.append(band)
    break

/datag/pipeline/AGBT21B_999_13/blc26/guppi_59470_06877_HIP113409_0086.rawspec.0000.h5


ValueError: invalid literal for int() with base 10: 'i'

In [ ]:
np.save('/datax/scratch/benjb/bl_nearby_stars/band_list.npy', band_list)

In [ ]:
df = pd.read_csv('/datax/scratch/benjb/bl_nearby_stars/star_cadences_catalog_with_receiver.csv')
dfc = df.iloc[np.where(df['Receiver'].values == 'Rcvr8_10')]
semesters = []
mjds = []
for i in range(len(dfc)):
    h5 = dfc['.h5 path'].values[i]
    s = h5.split('AGBT')[1][:3]
    mjd = h5.split('guppi_')[1][:5]
    semesters.append(s)
    mjds.append(float(mjd))
    #if s == '23B':
    #    print(mjd)
u, c = np.unique(semesters, return_counts=True)
plt.bar(u, c)
plt.yscale('log')
plt.show()
plt.hist(mjds, bins=100)
plt.axvspan(59976, 60156, lw=0, color='red', alpha=0.2)
plt.axvspan(60096, 60125, lw=0, color='k')
plt.xlabel('MJD')
plt.yscale('log')
plt.show()